In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

## Preprocessing

In [ ]:
df1 = pd.read_csv('Medium - Final.csv')
df2 = df1
df2

FileNotFoundError: ignored

In [ ]:
# Finding median of the 'Claps' column
# Here Claps is a metric of popularity
df2['Claps'].median()

In [ ]:
for row in range(df1.shape[0]):
  if df1['Claps'][row] < 99:
    df2['Claps'][row] = 0
  else :
    df2['Claps'][row] = 1
# The article with claps more than 99 are considered popular, else they are labelled unpopular


In [ ]:
for column in df2:
  print("The column '"+str(column)+"' is of the type '"+str(df2[column].dtypes)+"', has '"+str(len(df2[column].unique()))+"' unique values, and has '"+str(np.sum(pd.isnull(df2[column])))+"' null values")


In [ ]:
# converting Responses column(string) to integer
# errors = 'coerce' sets the invalid parsing as NaN
df2['Responses'] = pd.to_numeric(df2['Responses'],errors='coerce')

# Changing structure of the date variable
for row in range(df2.shape[0]):
  date = df2['Date'][row]
  try:
    date = date.split('-')
    df2.replace({'Date' : {df2['Date'][row]:int(date[0])*10000 + int(date[1])*100 + int(date[2])}},inplace=True)

  except:
    continue

In [ ]:
for column in df2:
  print("The column '"+str(column)+"' is of the type '"+str(df2[column].dtypes)+"', has '"+str(len(df2[column].unique()))+"' unique values, and has '"+str(np.sum(pd.isnull(df2[column])))+"' null values")


In [ ]:
# Input variables
# Here the url is ignored in the first column
X = np.column_stack((df2.iloc[:,5].values,df2.iloc[:,7:].values))
# Target variable
y = df2.iloc[:,6].values

In [ ]:
# One - hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3,-1])], remainder='passthrough')

X = np.array(ct.fit_transform(X))

In [ ]:
y[0]

In [ ]:
# Splitting data (train:validation:text :: 70:15:15)
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(X,y, test_size = 0.15, random_state = 0)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp,y_temp, test_size = 0.1764, random_state = 0)

In [ ]:
# Normalisation
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
# Normalising X_valid and Normalising X_test with the same mean and variance used to normalising X_train
X_test = sc.transform(X_test)
X_valid = sc.transform(X_valid)

## Logistic Rgression

In [ ]:
# Training
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

classifier.fit(X_train,y_train)

In [ ]:
# Accuracy and the confusion matrix
from sklearn.metrics import confusion_matrix
y_valid_pred = classifier.predict(X_valid)
cm = confusion_matrix(y_valid, y_valid_pred)

print('Accuracy:',classifier.score(X_valid,y_valid))
print('Confusion Matrix', cm)

In [ ]:
# Using different regularisation strengths

from sklearn.linear_model import LogisticRegression
from google.colab import widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix


accu1 = []
tb2 = widgets.TabBar([str(i) for i in ['C = 0.1','C = 0.5','C = 1','C = 1.5']])
C = [0.1,0.5,1,1.5]
for i in range(4):
  with tb2.output_to(i):
    # Splitting
    X_temp, X_test, y_temp, y_test = train_test_split(X,y, test_size = 0.15, random_state = 0)
    X_train, X_valid, y_train, y_valid = train_test_split(X_temp,y_temp, test_size = 0.1764, random_state = 0)

    # Normalisation
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_valid = sc.transform(X_valid)

    # Training
    classifier = LogisticRegression(C = C[i],tol = 1e-3)
    classifier.fit(X_train,y_train)

    # Predicting
    y_valid_pred = classifier.predict(X_valid)

    # Error Measurement
    cm = confusion_matrix(y_valid, y_valid_pred)
    accu1.append(classifier.score(X_valid,y_valid))
    print('Accuracy:',classifier.score(X_valid,y_valid))
    print('Confusion Matrix', cm)


# Here different tabs corresponds to different hyperparameters
# In this case it is the Regularisation parameter

In [ ]:
sb.lineplot(x = [0.1,0.5,1,1.5], y = accu1)
plt.xlabel('Regularisation Parameter',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy vs regularisation parameter(Logistic Reg.)',fontsize = 16);

## SVC

In [ ]:
# Training
from sklearn.svm import SVC

classifier = SVC(kernel='rbf')
classifier.fit(X_train,y_train)

In [ ]:
# Accuracy and the confusion matrix
from sklearn.metrics import confusion_matrix
y_valid_pred = classifier.predict(X_valid)
cm = confusion_matrix(y_valid, y_valid_pred)

print('Accuracy:',classifier.score(X_valid,y_valid))
print('Confusion Matrix', cm)

In [ ]:
# Using different regularisation strengths

from sklearn.svm import SVC
from google.colab import widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

accu2 = []
tb2 = widgets.TabBar([str(i) for i in ['C = 1','C = 5','C = 10','C = 20','C = 30']])
C = [1,5,10,20,30]
for i in range(5):
  with tb2.output_to(i):
    # Splitting
    X_temp, X_test, y_temp, y_test = train_test_split(X,y, test_size = 0.15, random_state = 0)
    X_train, X_valid, y_train, y_valid = train_test_split(X_temp,y_temp, test_size = 0.1764, random_state = 0)

    # Normalisation
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_valid = sc.transform(X_valid)

    # Training
    classifier = SVC(C = C[i])
    classifier.fit(X_train,y_train)

    # Predicting
    y_valid_pred = classifier.predict(X_valid)

    # Error Measurement
    cm = confusion_matrix(y_valid, y_valid_pred)
    accu2.append(classifier.score(X_valid,y_valid))
    print('Accuracy:',classifier.score(X_valid,y_valid))
    print('Confusion Matrix', cm)


# Here different tabs corresponds to different hyperparameters
# In this case it is the Regularisation parameter

In [ ]:
sb.lineplot(x = [1,5,10,20,30], y = accu2)
plt.xlabel('Regularisation Parameter',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy vs regularisation parameter(SVC)',fontsize = 16);

## Random Forest

In [ ]:
# Training
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators = 100, random_state = 0)
classifier.fit(X_train,y_train)

In [ ]:
# Accuracy and the confusion matrix
from sklearn.metrics import confusion_matrix
y_valid_pred = classifier.predict(X_valid)
cm = confusion_matrix(y_valid, y_valid_pred)

print('Accuracy:',classifier.score(X_valid,y_valid))
print('Confusion Matrix', cm)

In [ ]:
# Using different number of trees

from sklearn.ensemble import RandomForestClassifier
from google.colab import widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

accu3 = []
tb2 = widgets.TabBar([str(i) for i in ['n_estimators = 10','n_estimators = 50','n_estimators = 100','n_estimators = 200','n_estimators = 300']])
n_estimators = [10,50,100,200,300]
for i in range(5):
  with tb2.output_to(i):
    # Splitting
    X_temp, X_test, y_temp, y_test = train_test_split(X,y, test_size = 0.15, random_state = 0)
    X_train, X_valid, y_train, y_valid = train_test_split(X_temp,y_temp, test_size = 0.1764, random_state = 0)


    # Training
    classifier = RandomForestClassifier(n_estimators = n_estimators[i])
    classifier.fit(X_train,y_train)
    

    # Predicting
    y_valid_pred = classifier.predict(X_valid)

    # Error Measurement
    cm = confusion_matrix(y_valid, y_valid_pred)
    accu3.append(classifier.score(X_valid,y_valid))
    print('Accuracy:',classifier.score(X_valid,y_valid))
    print('Confusion Matrix', cm)


# Here different tabs corresponds to different hyperparameters
# In this case, it is the number of trees in the forest

In [ ]:
sb.lineplot(x = [10,50,100,200,300], y = accu3)
plt.xlabel('Number of Trees',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy vs regularisation parameter(R.F.)',fontsize = 16);

## NN

In [ ]:
import seaborn as sb
import torch
import torch.nn as nn

In [ ]:
# Setting up the data types and device to be used
dtype = torch.float
device = torch.device("cpu")

In [ ]:
# Dataset module for the dataset
class mydataset(torch.utils.data.Dataset):
  def __init__(self,input,labels):
    super(mydataset,self).__init__()
    self.x = torch.from_numpy(input)
    self.y = torch.from_numpy(labels)
    self.n_samples = self.x.shape[0]

  def __getitem__(self,index):
       return self.x[index], self.y[index]

  def __len__(self):
    return self.n_samples

In [ ]:
X_train = X_train.astype(None)
X_valid = X_valid.astype(None)
X_test = X_test.astype(None)

In [ ]:
train_data = mydataset(X_train,y_train)
valid_data = mydataset(X_valid,y_valid)
test_data = mydataset(X_test,y_test)

# We want to include the last batch as well so we will set drop_last as False
# We only need shuffle in the train data set as it help with the training
train_loader = torch.utils.data.DataLoader(dataset= train_data, batch_size= 50)
valid_loader = torch.utils.data.DataLoader(dataset= valid_data, batch_size= 50,
                                           shuffle = True, drop_last = False)
test_loader = torch.utils.data.DataLoader(dataset= test_data, batch_size= 50,
                                          shuffle = True, drop_last = False)

In [ ]:
input_size = 46
hidden_size = 4
num_classes = 2


class mymodel(nn.Module):
  def __init__(self,input_size, num_classes,hidden_size=4):
    super(mymodel, self).__init__()
    self.l1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size,num_classes)
  
  def forward(self,x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out
  
  
model = mymodel(input_size,num_classes,hidden_size)

In [ ]:
# loss and optimizer
learning_rate = 1e-3

criterion = nn.CrossEntropyLoss()
criterion2 = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
# learning loop
n_total_steps = len(train_loader)
num_epochs = 200

train_loss = []
train_accuracy = []
valid_loss = []
valid_accuracy = []

for epoch in range(num_epochs):
  for input, labels in train_loader:

    input = input.to(device) # to push the input to device
    labels = labels.to(device) # to push the labels to device

    # forward pass
    outputs = model.forward(input.float())
    loss = criterion(outputs, labels)
    
    # backward pass
    optimizer.zero_grad() # to empty the value in the gradient attribute
    loss.backward() # for backpropogation
    optimizer.step() # to update the parameters

  # Loss for training set
  with torch.no_grad():
    output_train = model(torch.from_numpy(X_train).float())
    loss_train = criterion(output_train,torch.from_numpy(y_train))

  # Accuracy for training set
  _, predict_train = torch.max(output_train,1)
  n_samples = torch.from_numpy(y_train).shape[0]
  n_correct = (predict_train == torch.from_numpy(y_train)).sum().item()

  train_loss.append(loss_train.item())
  train_accuracy.append(100 * n_correct / n_samples)

  # Loss for validation set
  with torch.no_grad():
    output_valid = model(torch.from_numpy(X_valid).float())
    loss_valid = criterion(output_valid,torch.from_numpy(y_valid))

  # Accuracy for validation set
  _, predict_valid = torch.max(output_valid,1) # not interested in the first value
  n_samples = torch.from_numpy(y_valid).shape[0]
  n_correct = (predict_valid == torch.from_numpy(y_valid)).sum().item()

  
  valid_loss.append(loss_valid.item())
  valid_accuracy.append(100 * n_correct / n_samples)

### 8

#### (a)

In [ ]:
sb.lineplot(x = range(1,201), y = train_loss)
sb.lineplot(x = range(1,201), y = valid_loss)
plt.legend(['Training Loss','Validation Loss'])
plt.xlabel('Number of epochs',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Training and Validation loss vs epoch',fontsize = 20);

#### (b)

In [ ]:
sb.lineplot(x = range(1,201), y = train_accuracy)
sb.lineplot(x = range(1,201), y = valid_accuracy)
plt.legend(['Training Accuracy','Validation Accuracy'])
plt.xlabel('Number of epochs',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Training and Validation Accuracy vs epoch',fontsize = 20);

#### (c)

In [ ]:
# Validation Loss vs Hidden layer size

#for hidden_size in [20,30,40,50,60]:
for hidden_size in [20,30,40,50,60]:
  new_model = mymodel(input_size,num_classes,hidden_size)

  # loss and optimizer
  learning_rate = 1e-3
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(new_model.parameters(), lr = learning_rate)

  # learning loop
  n_total_steps = len(train_loader)
  num_epochs = 100
  valid_loss = []

  for epoch in range(num_epochs):
    for input, labels in train_loader:
      input = input.to(device) # to push the input to device
      labels = labels.to(device) # to push the labels to device

      # forward pass
      outputs = new_model.forward(input.float())
      loss = criterion(outputs, labels)
    
    
      # backward pass
      optimizer.zero_grad() # to empty the value in the gradient attribute
      loss.backward() # for backpropogation
      optimizer.step() # to update the parameters


    # Loss for validation set
    with torch.no_grad():
      output_valid = new_model(torch.from_numpy(X_train).float())
      loss_valid = criterion(output_valid,torch.from_numpy(y_train))

  
    valid_loss.append(loss_valid.item())
    
  sb.lineplot(x = range(1,101), y = valid_loss)

plt.legend(['Layer size = 20','Layer size = 30','Layer size = 40','Layer size = 50',
            'Layer size = 60'])
plt.xlabel('Number of epochs',fontsize=16)
plt.ylabel('Validation Loss',fontsize=16)
plt.title('Validation loss vs Hidden Layer size',fontsize = 20);
# Graph shows that the neural network with 50 hidden layers has highest decline 
# in the validation loss (for a common number of epochs)

#### (d)

In [ ]:
# Validation Loss vs Learning rate


for learning_rate in [1e-5,1e-4,1e-3,1e-2,1e-1]:
  hidden_size = 50 # best layer size
  new_model = mymodel(input_size,num_classes,hidden_size)

  # loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(new_model.parameters(), lr = learning_rate)

  # learning loop
  n_total_steps = len(train_loader)
  num_epochs = 100
  valid_loss = []

  for epoch in range(num_epochs):
    for input, labels in train_loader:
      input = input.to(device) # to push the input to device
      labels = labels.to(device) # to push the labels to device

      # forward pass
      outputs = new_model.forward(input.float())
      loss = criterion(outputs, labels)
    
    
      # backward pass
      optimizer.zero_grad() # to empty the value in the gradient attribute
      loss.backward() # for backpropogation
      optimizer.step() # to update the parameters


    # Loss for validation set
    with torch.no_grad():
      output_valid = new_model(torch.from_numpy(X_train).float())
      loss_valid = criterion(output_valid,torch.from_numpy(y_train))

  
    valid_loss.append(loss_valid.item())
    #print('epoch: ',epoch+1,'/',num_epochs,'|| Valid loss: ',
    #      np.round(loss_valid.item(),4))
  
  sb.lineplot(x = range(1,101), y = valid_loss)

plt.legend(['Learning Rate = 1e-5','Learning Rate = 1e-4','Learning Rate = 1e-3',
            'Learning Rate = 1e-2','Learning Rate = 1e-1'])
plt.xlabel('Number of epochs',fontsize=16)
plt.ylabel('Validation Loss',fontsize=16)
plt.title('Validation loss vs Learning Rate',fontsize = 20);
# Graph shows that the neural network with learning rate as 1e-1 has highest 
# decline in the validation loss (for a common number of epochs)